## 数据回放测试，用于查找问题

In [ ]:
from chanlun.backtesting.backtest_klines import BackTestKlines
from chanlun import kcharts
from chanlun.cl_utils import query_cl_chart_config
from chanlun.exchange.exchange import *
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import ttk, messagebox
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from chanlun.exchange.exchange_db import ExchangeDB
import time
from chanlun import cl
import os
import webbrowser
from chanlun.backtesting.base import Strategy

In [27]:

import pandas as pd
from chanlun.cl_utils import query_cl_chart_config
from chanlun.exchange.exchange_db import ExchangeDB
from chanlun.cl_utils import *
def get_history(
    market: str = 'a',
    code: str = 'SHSE.000300',
    frequency: str = '1m',
    count: int = 0,
    ealry: bool = False, 
    start_date: str = "2025-05-16 09:30:00",
    end_date: str = "2025-05-16 15:01:00"
) -> pd.DataFrame:
    """
    获取历史K线数据并进行预处理
    
    参数:
        market (str): 市场标识符，默认为 'a' 表示沪深A股。
        code (str): 标的资产代码，默认为 'SHSE.000300'。
        frequency (str): 周期列表，默认为 '1m'。
        start_date (str): 开始日期和时间，默认为 "2025-05-16 09:30:00"。
        end_date (str): 结束日期和时间，默认为 "2025-05-16 15:01:00"。
        
    返回:
        pd.DataFrame: 包含 'open', 'high', 'low', 'close', 'volume' 的 DataFrame，索引为 datetime。
    """
    
    # 查询缠论配置
    cl_config = query_cl_chart_config(market, code)
    # print(cl_config)

    # 使用 ExchangeDB 读取数据库中的 K 线数据
    ex = ExchangeDB(market)
    # print("end_date",end_date)
    # 获取 K 线数据
    klines = ex.klines(code, frequency=frequency, start_date=start_date, end_date=end_date)
    # print(klines.iloc[-1])
    # 提取所需列并确保 'date' 列是 datetime 类型
    if count and not ealry:
        df = klines[['date', 'open', 'high', 'low', 'close', 'volume']][-count:]
    elif count and ealry:
        df = klines[['date', 'open', 'high', 'low', 'close', 'volume']][:count]
    else:
        df = klines[['date', 'open', 'high', 'low', 'close', 'volume']]
    df['date'] = pd.to_datetime(df['date'])

    # 设置 'date' 列为索引
    df.set_index('date', inplace=True)

    # 添加 timestamp 列（可选）
    df['timestamp'] = df.index

    return df

if __name__ == '__main__':
    df = get_history(market='a', code='SHSE.000300',count=300, frequency='5m', start_date="2025-05-16 09:30:00", end_date="2025-09-05 10:01:00")
    print(df.tail(100))

                               open      high       low     close  \
date                                                                
2025-09-03 09:45:00+08:00  4489.383  4502.363  4488.855  4501.591   
2025-09-03 09:50:00+08:00  4501.468  4506.270  4496.999  4498.075   
2025-09-03 09:55:00+08:00  4497.868  4498.103  4490.054  4491.505   
2025-09-03 10:00:00+08:00  4491.383  4492.109  4481.624  4482.450   
2025-09-03 10:05:00+08:00  4482.259  4485.343  4470.218  4472.397   
...                             ...       ...       ...       ...   
2025-09-05 09:40:00+08:00  4356.514  4362.810  4350.297  4361.757   
2025-09-05 09:45:00+08:00  4361.493  4368.051  4359.787  4360.284   
2025-09-05 09:50:00+08:00  4360.813  4377.915  4360.813  4376.865   
2025-09-05 09:55:00+08:00  4376.488  4382.581  4366.787  4382.574   
2025-09-05 10:00:00+08:00  4382.676  4391.763  4382.676  4391.677   

                                 volume                 timestamp  
date                              

In [28]:
df.to_csv("D:/tmp.csv")
a=pd.read_csv("D:/tmp.csv")

In [26]:
a

,date,open,high,low,close,volume,timestamp
0,2025-09-03 14:30:00+08:00,4460.050,4463.853,4460.050,4463.853,62492200.0,2025-09-03 14:30:00+08:00
1,2025-09-03 14:31:00+08:00,4464.325,4467.605,4464.325,4467.168,92011700.0,2025-09-03 14:31:00+08:00
2,2025-09-03 14:32:00+08:00,4467.189,4469.378,4464.958,4469.345,74907700.0,2025-09-03 14:32:00+08:00
3,2025-09-03 14:33:00+08:00,4469.865,4470.203,4463.675,4463.675,73433600.0,2025-09-03 14:33:00+08:00
4,2025-09-03 14:34:00+08:00,4463.454,4466.412,4463.454,4466.254,60146700.0,2025-09-03 14:34:00+08:00
...,...,...,...,...,...,...,...
295,2025-09-05 09:57:00+08:00,4384.457,4385.120,4384.001,4384.824,158644900.0,2025-09-05 09:57:00+08:00
296,2025-09-05 09:58:00+08:00,4384.572,4386.540,4384.118,4384.852,130958300.0,2025-09-05 09:58:00+08:00
297,2025-09-05 09:59:00+08:00,4385.203,4385.850,4383.005,4385.844,142983900.0,2025-09-05 09:59:00+08:00
298,2025-09-05 10:00:00+08:00,4385.986,4391.763,4385.793,4391.677,146820300.0,2025-09-05 10:00:00+08:00


In [ ]:
market = 'a'
code = 'SHSE.000300'
start_date = '2025-02-14 09:30:00'
end_date = '2025-02-24 13:01:00'
frequencys = ['1m']
ex = ExchangeDB(market) # 读取数据库中的k线数据
cl_config = query_cl_chart_config(market, code) # 读取缠论配置
# cl_config["to_file"] = f"E:\stock_history_charts\{code}_{frequencys[0]}_{start_date[:10]}_{end_date[:10]}.html"

In [ ]:
print(cl_config)

In [ ]:
# 回放的数据配置
# market = 'currency'
# code = 'ETH/USDT'
# start_date = '2023-02-02 14:30:00'
# end_date = '2023-02-02 15:00:00'
# frequencys = ['5m']



cl_config = query_cl_chart_config(market, code)

bk = BackTestKlines(market, start_date, end_date, frequencys, cl_config)
bk.init(code, frequencys[0])

In [ ]:
# 依次执行此单元格，检查缠论计算结果
while (bk.next()):
    cd = bk.get_cl_data(code, frequencys[0])
    # kcharts.render_charts(code, cd, config=cl_config)
    high_cd = cd
    klines_lv0 = high_cd.get_klines()
    last_k = high_cd.get_src_klines()[-1]
    pre_k = high_cd.get_src_klines()[-2]
    price = last_k.c
    xdzs = high_cd.get_last_xd_zs()

    xd_start = xdzs.lines[0]
    xd_zs_klines_lv0 = klines_lv0[xd_start.start.k.k_index:]
    zz_xd,zd_xd,zg_xd = Strategy.get_frvp(cd_klines=xd_zs_klines_lv0, use_close=False)
    bi = high_cd.get_bis()[-1]
    pre_bi = high_cd.get_bis()[-2]
    # 触发信号
    if (
        bi.type == "down"
        and bi.end.done
        and abs(bi.low - zg_xd) <= zg_xd * 0.0005
    ):
        # print(f"{xd_start},{xd_zs_klines_lv0[0]}")
        # print(f"{last_k.date}触发,{xd_start.start.k.index},{xd_zs_klines_lv0[0].date},{xd_zs_klines_lv0[-1].date},{price},{bi.low}")
        # 入场信号
        frvp_bi_start_index = pre_bi.start.k.k_index
        print(f"index： {frvp_bi_start_index}")
        pos_info = {}
        pos_info["frvp_bi_start_index"] = frvp_bi_start_index
        bi_zs_klines_lv0 = klines_lv0[frvp_bi_start_index:]
        print(pre_bi)
        print(bi_zs_klines_lv0[0])
        print(bi_zs_klines_lv0[-1])
        zz_bi,zd_bi,zg_bi = Strategy.get_frvp(cd_klines=bi_zs_klines_lv0, use_close=False)

In [ ]:
klines['date'][0]
# Timestamp('2025-02-13 10:54:00+0800', tz='Asia/Shanghai')

In [ ]:
klines['date'][0]
# Timestamp('2025-02-13 10:54:00+0800', tz='Asia/Shanghai')

In [ ]:
klines['date'][0]
# Timestamp('2025-02-13 10:54:00+0800', tz='Asia/Shanghai')

In [ ]:
klines['date'][0]
# Timestamp('2025-02-13 10:54:00+0800', tz='Asia/Shanghai')

In [ ]:
print(xd_start)
print(xd_zs_klines_lv0[0].date, xd_zs_klines_lv0[-1].date)
print(xd_start.start.k.index)
print(len(klines_lv0))
print(len(xd_zs_klines_lv0))
print(klines_lv0[0])
print(xd_zs_klines_lv0[0])

In [ ]:
cd2 = bk.get_cl_data(code, frequencys[0])
kcharts.render_charts(code, cd2, config=cl_config)

In [ ]:
klines = cd2.get_src_klines()
k1 = klines[-1]
k2 = klines[-2]
print(k1.date,k2.date,(k2.date-k1.date).days)
# datetime.(k2.date,k1.date)

In [ ]:
klines

In [ ]:
zss = cd.get_xd_zss()
zs = zss[-1]
print(zs)

In [ ]:
last_zs = cd.get_last_xd_zs()
print(last_zs)
for line in last_zs.lines:
    print(line)

In [ ]:
print(zs.type)

In [ ]:
print(cd.get_last_xd_zs())

In [ ]:
for line in zs.lines:
    print(line)

In [ ]:
bk.now_date

In [ ]:
print(cd.get_xds()[-1])